In [1]:
import numpy as np

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import glob
import pathlib

import PIL
import PIL.Image

import tensorflow as tf
from tensorflow.keras import regularizers

import matplotlib.pyplot as plt


2024-08-19 13:18:04.766688: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-19 13:18:04.778246: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-19 13:18:04.781357: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
model = tf.keras.models.load_model('/home/papafrita/Projects/skin_cancer_tool/Skin_cancer_diagnosis_tool/models/CNN_97_diagnosis_segmented.keras')

I0000 00:00:1724066286.404135    3031 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724066286.430702    3031 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724066286.430761    3031 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724066286.434758    3031 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724066286.434858    3031 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [15]:
val_dir = "/home/papafrita/Projects/skin_cancer_tool/Skin_cancer_diagnosis_tool/data/val_segmented_images/"
val_ds = tf.keras.utils.image_dataset_from_directory(
            val_dir,
            seed = 42,
            image_size = (450, 600),
            batch_size = 32
            )

class_names = val_ds.class_names

Found 1511 files belonging to 7 classes.


In [16]:
class_names

['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [17]:
# Function to one-hot encode the labels
def one_hot_encode_labels(image, label):
    return image, tf.one_hot(label, 7)

val_ds = val_ds.map(one_hot_encode_labels)

In [18]:
model.evaluate(val_ds)

48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - auc: 0.5327 - loss: 2.6678


[2.6587958335876465, 0.5465832352638245]

In [19]:
y_pred = model.predict(val_ds)


 2/48 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step

48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step


In [20]:
y_pred[0]

array([0.0963875 , 0.16750161, 0.2923849 , 0.0407771 , 0.1308577 ,
       0.24392626, 0.02816484], dtype=float32)

In [21]:
# Function to get the class name from a one-hot encoded label
def get_class_name(one_hot_label):
    class_index = tf.argmax(one_hot_label).numpy()
    return class_names[class_index]

# Function to get class names from a batch of one-hot encoded labels
def get_class_names_batch(one_hot_labels):
    class_indices = tf.argmax(one_hot_labels, axis=1).numpy()
    return [class_names[i] for i in class_indices]

# Example usage:
for images, labels in val_ds.take(1):  # Take one batch
    print("Batch shape:", labels.shape)
    print("Class names for this batch:", get_class_names_batch(labels))
    print("Class name for the first image:", get_class_name(labels[0]))


Batch shape: (32, 7)
Class names for this batch: ['nv', 'mel', 'nv', 'nv', 'nv', 'nv', 'nv', 'nv', 'bkl', 'bkl', 'nv', 'nv', 'mel', 'mel', 'bkl', 'nv', 'vasc', 'nv', 'bkl', 'bcc', 'bcc', 'nv', 'nv', 'bkl', 'nv', 'nv', 'nv', 'nv', 'bcc', 'nv', 'nv', 'bkl']
Class name for the first image: nv


In [34]:
def get_label_from_probabilities(probabilities, class_names):
    """
    Convert softmax probabilities to a class label.
    
    Args:
    probabilities (array-like): Softmax output probabilities
    class_names (list): List of class names in the order corresponding to the probabilities
    
    Returns:
    tuple: (predicted_label, confidence)
    """
    # Get the index of the highest probability
    predicted_index = np.argmax(probabilities)
    
    # Get the corresponding class name
    predicted_label = class_names[predicted_index]
    
    # Get the confidence (highest probability)
    confidence = probabilities[predicted_index]
    
    return predicted_label, confidence

def get_labels_from_batch_probabilities(batch_probabilities, class_names):
    """
    Convert a batch of softmax probabilities to class labels.
    
    Args:
    batch_probabilities (array-like): Batch of softmax output probabilities
    class_names (list): List of class names in the order corresponding to the probabilities
    
    Returns:
    list of tuples: [(predicted_label, confidence), ...]
    """
    return [get_label_from_probabilities(probs, class_names) for probs in batch_probabilities]

predicted_label, confidence = get_label_from_probabilities(y_pred[0], class_names)
print(f"Predicted label: {predicted_label}")
print(f"Confidence: {confidence:.2f}")

Predicted label: bkl
Confidence: 0.29


In [35]:
batch_results = get_labels_from_batch_probabilities(y_pred, class_names)
for i, (label, conf) in enumerate(batch_results):
    print(f"Sample {i+1}: Predicted label: {label}, Confidence: {conf:.2f}")

Sample 1: Predicted label: bkl, Confidence: 0.29
Sample 2: Predicted label: bkl, Confidence: 0.31
Sample 3: Predicted label: nv, Confidence: 0.38
Sample 4: Predicted label: bkl, Confidence: 0.29
Sample 5: Predicted label: bkl, Confidence: 0.31
Sample 6: Predicted label: bkl, Confidence: 0.34
Sample 7: Predicted label: bkl, Confidence: 0.32
Sample 8: Predicted label: nv, Confidence: 0.42
Sample 9: Predicted label: bkl, Confidence: 0.31
Sample 10: Predicted label: bkl, Confidence: 0.32
Sample 11: Predicted label: bkl, Confidence: 0.33
Sample 12: Predicted label: bkl, Confidence: 0.31
Sample 13: Predicted label: bkl, Confidence: 0.30
Sample 14: Predicted label: bkl, Confidence: 0.32
Sample 15: Predicted label: bkl, Confidence: 0.34
Sample 16: Predicted label: bkl, Confidence: 0.37
Sample 17: Predicted label: bkl, Confidence: 0.30
Sample 18: Predicted label: nv, Confidence: 0.31
Sample 19: Predicted label: bkl, Confidence: 0.33
Sample 20: Predicted label: bkl, Confidence: 0.29
Sample 21: P